# Importing Packages

In [1]:
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

# Scraping MVP Results Data

In [85]:
# Define base URL and year range
base_url = "https://www.basketball-reference.com/awards/awards_"
years = range(1981, 2025)  # Iterate through years 1981 to 2023 (inclusive)

# Create an empty list to store all DataFrames
all_data = []

# Loop through each year
for year in years:
    # Construct the URL for the specific year
    url = f"{base_url}{year}.html"

    # Download webpage content
    response = requests.get(url)
    html_content = response.content

    # Parse HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the table and extract data
    table = soup.find('table')
    table_data = []
    rows = table.find_all('tr')
    for row in rows:
        row_data = []
        cells = row.find_all(['td', 'th'])
        for cell in cells:
            cell_text = cell.get_text(strip=True)
            row_data.append(cell_text)
        table_data.append(row_data)

    # Create a DataFrame for this year's data (optional: add year as a column)
    df = pd.DataFrame(table_data[1:])  # Skip the first row (assuming header) 
    df['Year'] = year  # Add a column with the year (uncomment if needed)

    # Append the DataFrame for this year to the all_data list
    all_data.append(df)
    #print(url)
    time.sleep(3)
# Combine all DataFrames into a single DataFrame (optional)
final_df = pd.concat(all_data, ignore_index=True)  # Concatenate all DataFrames vertically

# Print or use the final DataFrame (final_df) for further processing
#print(final_df)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,Year


In [87]:
#change headers, select columns, create MVP column
new_header = final_df.iloc[0]
awards_df = final_df[1:]
awards_df.columns = new_header
awards_df.columns = awards_df.columns.astype(str)
awards_df = awards_df.rename(columns={'1981': 'Year'})
awards_final = awards_df[['Rank','Player','Age','Tm','Share','Year']]
def mvp(Rank):
    if Rank == '1':
        return 1
    else:
        return 0
awards_final['MVP'] = awards_final['Rank'].apply(mvp)
awards_final

C:\Users\drawn\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Rank,Player,Age,Tm,Share,Year,MVP
1,1,Julius Erving,30,PHI,0.658,1981,1
2,2,Larry Bird,24,BOS,0.613,1981,0
3,3,Kareem Abdul-Jabbar,33,LAL,0.414,1981,0
4,4,Moses Malone,25,HOU,0.261,1981,0
5,5,George Gervin,28,SAS,0.120,1981,0
...,...,...,...,...,...,...,...
749,5,Jalen Brunson,27,NYK,0.143,2024,0
750,6,Jayson Tatum,25,BOS,0.087,2024,0
751,7,Anthony Edwards,22,MIN,0.018,2024,0
752,8,Domantas Sabonis,27,SAC,0.003,2024,0


In [88]:
# Define base URL and year range
base_url = "https://www.basketball-reference.com/leagues/NBA_"
years = range(1981, 2025)  # Iterate through years 1981 to 2023 (inclusive)

# Create an empty list to store all DataFrames
all_data = []

# Loop through each year
for year in years:
    # Construct the URL for the specific year
    url = f"{base_url}{year}_per_game.html"

    # Download webpage content
    response = requests.get(url)
    html_content = response.content

    # Parse HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the table and extract data
    table = soup.find('table')
    table_data = []
    rows = table.find_all('tr')
    for row in rows:
        row_data = []
        cells = row.find_all(['td', 'th'])
        for cell in cells:
            cell_text = cell.get_text(strip=True)
            row_data.append(cell_text)
        table_data.append(row_data)

    # Create a DataFrame for this year's data (optional: add year as a column)
    df = pd.DataFrame(table_data[1:])  # Skip the first row (assuming header) 
    df['Year'] = year  # Add a column with the year (uncomment if needed)

    # Append the DataFrame for this year to the all_data list
    all_data.append(df)
    #print(url)
    time.sleep(3)
# Combine all DataFrames into a single DataFrame (optional)
players_df = pd.concat(all_data, ignore_index=True)  # Concatenate all DataFrames vertically

# Print or use the final DataFrame (final_df) for further processing
#print(players_df)

In [89]:
w = players_df[players_df['Year'] > 2024]
w

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,Year


In [13]:
#create headers, select rows for traded players, select columns, remove '*'
players_header = ['Rk','Player','Age','Tm','Pos','G','GS','MP','FG','FGA','FG%',
                  '3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','Notes','Year']
players_df.columns = players_header

players_df['G'] = pd.to_numeric(players_df['G'], errors='coerce')

idx = players_df.groupby(['Player', 'Year', 'Age'])['G'].idxmax()

filtered_df = players_df.loc[idx]
players_final = filtered_df[['Player','Year','Pos','Age','Tm','G','MP','FG','FGA','FG%','3P',
                            '3PA','3P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST',
                            'STL','BLK','TOV','PF','PTS']]
players_final['Player'] = players_final['Player'].str.replace('*', '', regex=False)
players_final

C:\Users\drawn\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.
C:\Users\drawn\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Player,Year,Pos,Age,Tm,G,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
2052.0,A.C. Green,1986.0,PF,22,LAL,82.0,18.8,2.5,4.7,.539,...,.611,2.0,2.7,4.6,0.7,0.6,0.6,1.2,2.8,6.4
2339.0,A.C. Green,1987.0,PF,23,LAL,79.0,28.4,4.0,7.4,.538,...,.780,2.7,5.1,7.8,1.1,0.9,1.0,1.3,2.2,10.8
2720.0,A.C. Green,1988.0,PF,24,LAL,82.0,32.1,3.9,7.8,.503,...,.773,3.0,5.7,8.7,1.1,1.1,0.5,1.5,2.5,11.4
3126.0,A.C. Green,1989.0,PF,25,LAL,82.0,30.6,4.9,9.2,.529,...,.786,3.1,5.9,9.0,1.3,1.1,0.7,1.5,2.1,13.3
3568.0,A.C. Green,1990.0,PF,26,LAL,82.0,33.0,4.7,9.8,.478,...,.751,3.2,5.5,8.7,1.1,0.8,0.6,1.4,2.5,12.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9655.0,Željko Rebrača,2002.0,C,29,DET,74.0,15.9,2.6,5.1,.505,...,.771,1.1,2.8,3.9,0.5,0.4,1.0,1.1,2.6,6.9
10142.0,Željko Rebrača,2003.0,C,30,DET,30.0,16.3,2.7,4.8,.552,...,.792,0.9,2.2,3.1,0.3,0.2,0.6,1.0,2.6,6.6
10807.0,Željko Rebrača,2004.0,C,31,2TM,24.0,11.4,1.4,3.2,.442,...,.767,1.0,1.5,2.4,0.3,0.2,0.5,0.7,2.2,3.8
11313.0,Željko Rebrača,2005.0,C,32,LAC,58.0,16.0,2.3,4.0,.568,...,.859,0.8,2.3,3.2,0.4,0.2,0.7,0.8,2.2,5.8


In [14]:
# Define base URL and year range
base_url = "https://www.basketball-reference.com/leagues/NBA_"
years = range(1981, 2026)  # Iterate through years 1981 to 2023 (inclusive)

# Create an empty list to store all DataFrames
all_data = []

# Loop through each year
for year in years:
    # Construct the URL for the specific year
    url = f"{base_url}{year}_advanced.html"

    # Download webpage content
    response = requests.get(url)
    html_content = response.content

    # Parse HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the table and extract data
    table = soup.find('table')
    table_data = []
    rows = table.find_all('tr')
    for row in rows:
        row_data = []
        cells = row.find_all(['td', 'th'])
        for cell in cells:
            cell_text = cell.get_text(strip=True)
            row_data.append(cell_text)
        table_data.append(row_data)

    # Create a DataFrame for this year's data (optional: add year as a column)
    df = pd.DataFrame(table_data[1:])  # Skip the first row (assuming header) 
    df['Year'] = year  # Add a column with the year (uncomment if needed)

    # Append the DataFrame for this year to the all_data list
    all_data.append(df)
    #print(url)
    time.sleep(3)
# Combine all DataFrames into a single DataFrame (optional)
advanced_df = pd.concat(all_data, ignore_index=True)  # Concatenate all DataFrames vertically

# Print or use the final DataFrame (final_df) for further processing
#print(advanced_df)

In [64]:
advanced_df

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,null2,OBPM,DBPM,BPM,VORP,Year
0,1,Adrian Dantley,25,UTA,SF,80.0,,3417,24.3,.622,...,12.3,1.3,13.6,.191,5.7,-1.4,4.3,5.4,"MVP-12,AS,NBA2",1981
1,2,Moses Malone,25,HOU,C,80.0,,3245,25.1,.585,...,9.5,4.2,13.7,.202,4.5,-1.2,3.3,4.3,"MVP-4,AS,NBA2",1981
2,3,Larry Bird,24,BOS,PF,82.0,82,3239,19.9,.528,...,4.7,6.1,10.8,.160,2.5,1.8,4.3,5.2,"MVP-2,AS,NBA1",1981
3,4,Mike Mitchell,25,CLE,SF,82.0,82,3194,17.5,.513,...,5.0,1.1,6.0,.091,1.9,-2.2,-0.3,1.3,"MVP-24,AS",1981
4,5,Michael Ray Richardson,25,NYK,SG,79.0,79,3175,17.1,.511,...,2.2,4.7,6.9,.104,1.4,1.9,3.2,4.2,"MVP-17,AS",1981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23563,569,Jalen Crutcher,24,NOP,PG,1.0,0,3,-12.6,.000,...,0.0,0.0,0.0,-0.334,-18.5,-7.8,-26.2,0.0,,2024
23564,570,Dmytro Skapintsev,25,NYK,C,2.0,0,2,-19.3,.000,...,0.0,0.0,0.0,-0.483,-16.0,-9.8,-25.9,0.0,,2024
23565,571,Justin Jackson,28,MIN,SF,2.0,0,1,0.0,,...,0.0,0.0,0.0,.031,-6.3,-1.2,-7.5,0.0,,2024
23566,572,Javonte Smart,24,PHI,PG,1.0,0,1,0.0,,...,0.0,0.0,0.0,.005,-6.2,-2.1,-8.3,0.0,,2024


In [65]:
advanced_header = ['Rk','Player','Age','Tm','Pos','G','MP','PER','TS%','3PAr','FTr',
                  'ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','null','OWS','DWS','WS','WS/48','null2','OBPM','DBPM','BPM','VORP','Year']
advanced_df.columns = advanced_header

advanced_df['G'] = pd.to_numeric(advanced_df['G'], errors='coerce')

idx = advanced_df.groupby(['Player', 'Year', 'Age'])['G'].idxmax()
filtered_df_a = advanced_df.loc[idx]
advanced_final = filtered_df_a[['Player','Age','USG%','WS/48','OBPM','DBPM','BPM','Year']]
advanced_final

C:\Users\drawn\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


,Player,Age,USG%,WS/48,OBPM,DBPM,BPM,Year
1994.0,A.C. Green,22,17.7,.103,0.1,-1.6,0.2,1986.0
2296.0,A.C. Green,23,12.5,.163,0.1,1.0,1.7,1987.0
2644.0,A.C. Green,24,12.9,.144,-0.1,0.3,1.5,1988.0
3088.0,A.C. Green,25,11.5,.179,-0.2,1.6,2.3,1989.0
3510.0,A.C. Green,26,10.7,.137,-0.9,-0.6,1.0,1990.0
...,...,...,...,...,...,...,...,...
9679.0,Željko Rebrača,29,15.7,.134,0.6,-2.3,-0.1,2002.0
10272.0,Željko Rebrača,30,14.9,.133,0.2,-2.6,-0.1,2003.0
10876.0,Željko Rebrača,31,15.9,.097,0.6,-3.9,-0.1,2004.0
11334.0,Željko Rebrača,32,15.3,.122,0.4,-1.3,0.2,2005.0


In [21]:
print(players_final.index.duplicated().any())  # Check duplicates in players_final
print(advanced_final.index.duplicated().any())

players_final = players_final.reset_index(drop=True)
advanced_final = advanced_final.reset_index(drop=True)

True
True


In [66]:

advanced_final['Player'] = players_final['Player'].str.replace('*', '', regex=False)
advanced_final

C:\Users\drawn\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Player,Age,USG%,WS/48,OBPM,DBPM,BPM,Year
1994.0,Brandon Rush,22,17.7,.103,0.1,-1.6,0.2,1986.0
2296.0,Buck Williams,23,12.5,.163,0.1,1.0,1.7,1987.0
2644.0,Cedric Ball,24,12.9,.144,-0.1,0.3,1.5,1988.0
3088.0,Chris Mills,25,11.5,.179,-0.2,1.6,2.3,1989.0
3510.0,Corey Williams,26,10.7,.137,-0.9,-0.6,1.0,1990.0
...,...,...,...,...,...,...,...,...
9679.0,Jordan Farmar,29,15.7,.134,0.6,-2.3,-0.1,2002.0
10272.0,Kelly Tripucka,30,14.9,.133,0.2,-2.6,-0.1,2003.0
10876.0,Klay Thompson,31,15.9,.097,0.6,-3.9,-0.1,2004.0
11334.0,Larry Robinson,32,15.3,.122,0.4,-1.3,0.2,2005.0


In [23]:
base_url = "https://www.basketball-reference.com/leagues/NBA_"
years = range(1981, 2026)


all_data = []

# Loop through each year
for year in years:
    url = f"{base_url}{year}.html"

    response = requests.get(url)
    html_content = response.content

    soup = BeautifulSoup(html_content, "html.parser")

    table = soup.find('table', id = 'advanced-team')
    table_data = []
    rows = table.find_all('tr')
    for row in rows:
        row_data = []
        cells = row.find_all(['td', 'th'])
        for cell in cells:
            cell_text = cell.get_text(strip=True)
            row_data.append(cell_text)
        table_data.append(row_data)

    df = pd.DataFrame(table_data[1:])
    df['Year'] = year
    all_data.append(df)
    #print(url)
    time.sleep(3)

teams_df = pd.concat(all_data, ignore_index=True)


#print(teams_df)    

In [24]:
#change headers, filter rows, select columns, replace '*'
teams_header = ['Rk','Team','Age','W','L','PW','PL','MOV','SOS','SRS','ORtg','DRtg','NRtg','Pace','FTr','3PAr','TS%','null','eFG%'
                ,'TOV%','ORB%','FT/FGA','null3','eFG%','TOV%','DRB%','FT/FGA','null2','Arena','Attend.','Attend./G','Year']
teams_df.columns = teams_header
a = teams_df[(teams_df.Team != 'Team') & (teams_df.Team != 'League Average')]
a = a[['Team','Year','W','L','NRtg']]
a['Team'] = a['Team'].str.replace('*', '', regex=False)

In [25]:
# Define the list of team names
team_names = ['Philadelphia 76ers', 'Milwaukee Bucks', 'Boston Celtics', 'Phoenix Suns', 'Los Angeles Lakers',
              'San Antonio Spurs', 'Chicago Bulls', 'New York Knicks', 'Indiana Pacers', 'Portland Trail Blazers',
              'Houston Rockets', 'Washington Bullets', 'Kansas City Kings', 'Denver Nuggets',
              'Golden State Warriors', 'San Diego Clippers', 'Seattle SuperSonics', 'Atlanta Hawks',
              'Cleveland Cavaliers', 'New Jersey Nets', 'Utah Jazz', 'Detroit Pistons', 'Dallas Mavericks',
              'Los Angeles Clippers', 'Sacramento Kings', 'Charlotte Hornets', 'Miami Heat',
              'Minnesota Timberwolves', 'Orlando Magic', 'Toronto Raptors', 'Vancouver Grizzlies',
              'Washington Wizards', 'Memphis Grizzlies', 'New Orleans Hornets', 'Charlotte Bobcats',
              'New Orleans/Oklahoma City Hornets', 'Oklahoma City Thunder', 'Brooklyn Nets', 'New Orleans Pelicans']

# Define the corresponding three-letter abbreviations
abbreviations = ['PHI', 'MIL', 'BOS', 'PHO', 'LAL', 'SAS', 'CHI', 'NYK', 'IND', 'POR', 'HOU', 'WSB', 'KCK', 'DEN',
                 'GSW', 'SDC', 'SEA', 'ATL', 'CLE', 'NJN', 'UTA', 'DET', 'DAL', 'LAC', 'SAC', 'CHA', 'MIA', 'MIN',
                 'ORL', 'TOR', 'VAN', 'WAS', 'MEM', 'NOH', 'CHA', 'NOK', 'OKC', 'BRK', 'NOP']

# Create a DataFrame
joining_df = pd.DataFrame({'Team Name': team_names, 'Abbreviation': abbreviations})

In [26]:
#join team data with team abbreviations so I can join with other tables
teams_final = pd.merge(a, joining_df, left_on='Team', right_on='Team Name', how='left')
teams_final = teams_final[['Team','Abbreviation','Year','W','L','NRtg']]
teams_final['NRtg'] = pd.to_numeric(teams_final['NRtg'], errors='coerce')

In [82]:
print(awards_final)
print(advanced_final)
print(players_final)
print(teams_final)
advanced_final.dropna(subset=['Year'], inplace=True)
advanced_final['Year'] = advanced_final['Year'].astype(int)
players_final['Year'] = players_final['Year'].astype(int)

0   Rank               Player Age   Tm  Share  Year  MVP
1      1        Julius Erving  30  PHI  0.658  1981    1
2      2           Larry Bird  24  BOS  0.613  1981    0
3      3  Kareem Abdul-Jabbar  33  LAL  0.414  1981    0
4      4         Moses Malone  25  HOU  0.261  1981    0
5      5        George Gervin  28  SAS  0.120  1981    0
..   ...                  ...  ..  ...    ...   ...  ...
749    5        Jalen Brunson  27  NYK  0.143  2024    0
750    6         Jayson Tatum  25  BOS  0.087  2024    0
751    7      Anthony Edwards  22  MIN  0.018  2024    0
752    8     Domantas Sabonis  27  SAC  0.003  2024    0
753    9         Kevin Durant  35  PHO  0.001  2024    0

[753 rows x 7 columns]
                 Player Age  USG% WS/48  OBPM  DBPM   BPM  Year
1994.0     Brandon Rush  22  17.7  .103   0.1  -1.6   0.2  1986
2296.0    Buck Williams  23  12.5  .163   0.1   1.0   1.7  1987
2644.0      Cedric Ball  24  12.9  .144  -0.1   0.3   1.5  1988
3088.0      Chris Mills  25  11.5  .

C:\Users\drawn\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\drawn\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ValueError: Cannot convert non-finite values (NA or inf) to integer

In [83]:
filtered_df = advanced_final[advanced_final['Year'] > 2024]
filtered_df

,Player,Age,USG%,WS/48,OBPM,DBPM,BPM,Year


In [77]:
#join together player, team, and award data
players_teams = pd.merge(players_final, teams_final, left_on=['Tm','Year'], right_on=['Abbreviation','Year'], how='left')
players_teams_awards = pd.merge(players_teams, awards_final, left_on=['Player','Year', 'Age'], right_on=['Player','Year','Age'], how='left')


In [78]:
players_teams_awards_advanced = pd.merge(players_teams_awards, advanced_final, on=['Player','Year','Age'], how='left')
players_teams_awards_advanced

,Player,Year,Pos,Age,Tm_x,G,MP,FG,FGA,FG%,...,NRtg,Rank,Tm_y,Share,MVP,USG%,WS/48,OBPM,DBPM,BPM
0,A.C. Green,1986.0,PF,22,LAL,82.0,18.8,2.5,4.7,.539,...,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.C. Green,1987.0,PF,23,LAL,79.0,28.4,4.0,7.4,.538,...,9.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A.C. Green,1988.0,PF,24,LAL,82.0,32.1,3.9,7.8,.503,...,5.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A.C. Green,1989.0,PF,25,LAL,82.0,30.6,4.9,9.2,.529,...,7.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.C. Green,1990.0,PF,26,LAL,82.0,33.0,4.7,9.8,.478,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19218,Željko Rebrača,2002.0,C,29,DET,74.0,15.9,2.6,5.1,.505,...,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19219,Željko Rebrača,2003.0,C,30,DET,30.0,16.3,2.7,4.8,.552,...,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19220,Željko Rebrača,2004.0,C,31,2TM,24.0,11.4,1.4,3.2,.442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19221,Željko Rebrača,2005.0,C,32,LAC,58.0,16.0,2.3,4.0,.568,...,-0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#select columns
players_teams_awards_advanced[['MVP','NRtg','Share']] = players_teams_awards_advanced[['MVP','NRtg','Share']].fillna(0)
columns_to_drop = ['Tm_x', 'FG', 'FG%', '3P', 'FT', 'DRB', 'Abbreviation', 'Rank', 'Tm_y']
players_teams_awards_advanced = players_teams_awards_advanced.drop(columns=columns_to_drop)

In [30]:
#creating new features(team win%, % of games played, prev mvps won)
players_teams_awards_advanced['W'] = pd.to_numeric(players_teams_awards_advanced['W'], errors='coerce')
players_teams_awards_advanced['L'] = pd.to_numeric(players_teams_awards_advanced['L'], errors='coerce')
#team win percent
players_teams_awards_advanced['team_win_pct'] = np.where(
    pd.isna(players_teams_awards_advanced['W']) | pd.isna(players_teams_awards_advanced['L']),  # Check if either is NaN
    0.5,  # Assign 0.5 if either is NaN
    players_teams_awards_advanced['W'] / (players_teams_awards_advanced['W'] + players_teams_awards_advanced['L'])  # Perform the calculation if both are not NaN
)
#percent of team games played

players_teams_awards_advanced['pct_of_games'] = players_teams_awards_advanced['G'] / (players_teams_awards_advanced['W'] + players_teams_awards_advanced['L'])

# Compute the median of the new column, excluding NaN values
median_value = players_teams_awards_advanced['pct_of_games'].median()

#Replace NaN values in the new column with the computed median
players_teams_awards_advanced['pct_of_games'] = players_teams_awards_advanced['pct_of_games'].fillna(median_value)

#previous mvps won
# Step 3: Compute the cumulative sum of MVPs for each player
players_teams_awards_advanced['mvp_cumsum'] = players_teams_awards_advanced.groupby('Player')['MVP'].cumsum()

# Step 4: Shift the cumulative sum by one year for each player to reflect previous wins only
players_teams_awards_advanced['pre_mvps'] = players_teams_awards_advanced.groupby('Player')['mvp_cumsum'].shift(fill_value=0)


In [31]:
players_teams_awards_advanced['MP'] = pd.to_numeric(players_teams_awards_advanced['MP'], errors='coerce')
players_teams_awards_advanced = players_teams_awards_advanced[(players_teams_awards_advanced.pct_of_games >= .50)] 


In [33]:
data_2024 = players_teams_awards_advanced[players_teams_awards_advanced['Year'] == 2025]
data_2024 = data_2024[(data_2024.pct_of_games >= .79) & (data_2024.MP >= 20)]
historic_data = players_teams_awards_advanced[players_teams_awards_advanced['Year'] != 2025]
data_2024 = data_2024.reset_index(drop=True)
historic_data = historic_data.reset_index(drop=True)
Y = historic_data['Share']
X = historic_data[['PTS','team_win_pct','AST',
                   'TOV','NRtg','FTA','WS/48','DBPM','OBPM','pre_mvps','USG%']]

for column in X.columns:
    X[column] = pd.to_numeric(X[column], errors='coerce')
    
X = X.fillna(0)

C:\Users\drawn\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=15)


In [35]:
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=15)
gbr.fit(X_train, y_train)

predictions = gbr.predict(X_test)

# Evaluate the model, e.g., using mean squared error or another suitable metric for your problem
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.002018181963091451


In [36]:
feature_importances = gbr.feature_importances_
# Get feature names and their importance scores
feature_names = X_train.columns
feature_scores = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort the features by importance
feature_scores = feature_scores.sort_values(by='Importance', ascending=False)

print(feature_scores)

         Feature  Importance
0            PTS    0.414380
1   team_win_pct    0.361391
5            FTA    0.062772
2            AST    0.060559
4           NRtg    0.056835
9       pre_mvps    0.023348
3            TOV    0.020715
6          WS/48    0.000000
7           DBPM    0.000000
8           OBPM    0.000000
10          USG%    0.000000


In [37]:

X_2024 = data_2024[['PTS','team_win_pct', 'AST',
                    'TOV','NRtg','FTA','WS/48','DBPM','OBPM','pre_mvps','USG%']]
for column in X_2024.columns:
    X_2024[column] = pd.to_numeric(X_2024[column], errors='coerce')
    
X_2024 = X_2024.fillna(0)
predictions_2024 = gbr.predict(X_2024)


ValueError: Found array with 0 sample(s) (shape=(0, 11)) while a minimum of 1 is required.

In [93]:
predictions_2024 = pd.DataFrame(predictions_2024, columns=['Pred'])
result = data_2024.join(predictions_2024)
result[['Player','Pred']].sort_values('Pred', ascending = False).head(30)


,Player,Pred
104,Nikola Jokić,0.834297
91,Luka Dončić,0.611449
47,Giannis Antetokounmpo,0.436349
117,Shai Gilgeous-Alexander,0.413232
68,Jayson Tatum,0.199049
128,Victor Wembanyama,0.124542
67,Jaylen Brown,0.114505
83,Kevin Durant,0.060445
35,Devin Booker,0.059468
60,Jalen Brunson,0.044616


In [68]:
predictions = gbr.predict(X)
predictions = pd.DataFrame(predictions, columns = ['Pred'])
overall_results = historic_data.join(predictions)

max_share_per_year = overall_results.groupby('Year')['Pred'].transform(max)
overall_results['pred_mvp'] = (overall_results['Pred'] == max_share_per_year).astype(int)

mvp_winners = overall_results[overall_results['MVP'] == 1]

In [92]:
mvp_winners
overall_results[overall_results['Year'] == 2023].sort_values('Pred', ascending = False).head(20)
#overall_results.sort_values('Pred', ascending = False).head(10)

,Player,Year,Pos,Age,G,MP,FGA,3PA,3P%,eFG%,...,WS/48,OBPM,DBPM,BPM,team_win_pct,pct_of_games,mvp_cumsum,pre_mvps,Pred,pred_mvp
6749,Joel Embiid,2023.0,C,28,66.0,34.6,20.1,3.0,.330,.573,...,.259,6.8,2.3,9.2,0.658537,0.804878,1.0,0.0,0.888984,1
10164,Nikola Jokić,2023.0,C,27,69.0,33.7,14.8,2.2,.383,.660,...,.308,8.5,4.5,13.0,0.646341,0.841463,2.0,2.0,0.696285,0
4961,Giannis Antetokounmpo,2023.0,PF,28,63.0,32.1,20.3,2.7,.275,.572,...,.204,5.8,2.7,8.5,0.707317,0.768293,2.0,2.0,0.615057,0
6237,Jayson Tatum,2023.0,SF,24,74.0,36.9,21.1,9.3,.350,.543,...,.185,4.8,0.7,5.5,0.695122,0.902439,0.0,0.0,0.349516,0
13992,Shai Gilgeous-Alexander,2023.0,PG,24,68.0,35.5,20.3,2.5,.345,.531,...,.226,5.7,1.5,7.3,0.487805,0.829268,0.0,0.0,0.116048,0
6594,Jimmy Butler,2023.0,PF,33,64.0,33.4,13.9,1.6,.350,.560,...,.277,6.7,2.0,8.7,0.536585,0.780488,0.0,0.0,0.115308,0
8629,Luka Dončić,2023.0,PG,23,66.0,36.2,22.0,8.2,.342,.560,...,.204,7.6,1.4,8.9,0.463415,0.804878,0.0,0.0,0.094466,0
654,Anthony Davis,2023.0,C,29,56.0,34.0,17.2,1.3,.257,.573,...,.226,4.6,1.7,6.3,0.524390,0.682927,0.0,0.0,0.092276,0
3915,Donovan Mitchell,2023.0,SG,26,68.0,35.8,20.6,9.3,.386,.572,...,.176,5.6,0.6,6.3,0.621951,0.829268,0.0,0.0,0.078910,0
3839,Domantas Sabonis,2023.0,C,26,79.0,34.6,11.9,1.1,.373,.632,...,.221,4.1,1.8,5.8,0.585366,0.963415,0.0,0.0,0.060136,0


In [73]:
diff = overall_results.groupby('Year')['Pred'].apply(lambda x: x.nlargest(2).diff().abs().min()).reset_index()


diff = diff.sort_values(by='Pred')
diff

,Year,Pred
8,1989.0,0.003592
17,1998.0,0.004158
21,2002.0,0.010449
1,1982.0,0.072214
25,2006.0,0.078074
41,2022.0,0.082683
24,2005.0,0.103403
16,1997.0,0.119416
27,2008.0,0.168433
38,2019.0,0.172047
